<img width="8%" alt="GitHub.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/GitHub.png" style="border-radius: 15%">

# GitHub - Get comments from issue
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=GitHub+-+Get+comments+from+issue:+Error+short+description">Bug report</a>

**Tags:** #github #comments #issue #automation #snippet

**Author:** [Varsha Kumar](https://www.linkedin.com/in/varsha-kumar-590466305/)

**Last update:** 2024-06-05 (Created: 2024-06-04)

**Description:** This notebook allows users to retrieve the comments from an issue.

## Input

### Import libraries

In [ ]:
import requests
import re
import pandas as pd
import naas_python

### Setup variables
- `github_token`: personal token creates
- `issue_url`: link to the chosen issue

In [ ]:
github_token = naas_python.secret.get("GITHUB_TOKEN").value
issue_url = "https://github.com/pola-rs/polars/issues/16661"

## Model


### Get issues comments

In [ ]:
def extract_repo_details(issue_url):
    # Extract owner, repo name, and issue number from the issue URL
    match = re.match(r'https://github\.com/(?P<owner>[^/]+)/(?P<repo>[^/]+)/issues/(?P<number>\d+)', issue_url)
    if not match:
        raise ValueError("Invalid issue URL")
    return match.group('owner'), match.group('repo'), match.group('number')

def get_issue_comments(issue_url):
    # Extract repo details
    repo_owner, repo_name, issue_number = extract_repo_details(issue_url)
    
    # GitHub API URL to get issue comments
    api_url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/issues/{issue_number}/comments"
    headers = {
        'Authorization': f'token {github_token}',
        'Accept': 'application/vnd.github.v3+json'
    }

    response = requests.get(api_url, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch comments: {response.status_code}")
        return []
    
comments = get_issue_comments(issue_url)

## Output


### Display result

In [ ]:
data = []
if len(comments) > 0:
    for comment in comments:
        comment_id = comment['id']
        comment_body = comment['body']
        comment_created_at = comment['created_at']
        user_id = comment['user']['id']
        comment_user_login = comment['user']['login']
        reaction_count = comment['reactions']['total_count']

        data.append({
            "COMMENT_ID": comment_id,
            "CONTENT": comment_body,
            "DATE_TIME": comment_created_at,
            "CREATOR_LOGIN": comment_user_login,
            "USER_ID": user_id,
            "REACTIONS_COUNT": reaction_count
        }) 
df = pd.DataFrame(data)
df